In [2]:
from pathlib import Path
from copy import copy
from openpyxl import load_workbook, Workbook
from openpyxl.styles import Font

In [ ]:
# ── 1. Ask for the file ────────────────────────────────────────────
src = Path(input("👉 Enter full path to the Excel file: ").strip('"')).expanduser()
if not src.is_file():
    raise FileNotFoundError(src)
print(f"✓ Found: {src}")

In [ ]:
# ── 2. Settings ────────────────────────────────────────────────────
TARGET_LANGS = {"Spanish", "German", "French", "Portuguese", "Italian"}
RED_RGB      = "FF0000"
hits_total   = mex_total = merged_rows = 0

wb = load_workbook(src)

In [ ]:
# ── 3. Process every sheet ────────────────────────────────────────
for ws in wb.worksheets:
    # 3-a) header list
    header = [str(c.value).strip() if c.value else "" for c in ws[1]]

    # 3-b) collect usage-status partners for language columns
    usage_cols = [i for i, h in enumerate(header, 1) if h.lower() == "usage status"]
    lang_to_usage = {i: next((u for u in usage_cols if u > i), None)
                     for i, h in enumerate(header, 1) if h in TARGET_LANGS}

    # 3-c) rename headers (only if still needed)
    for c in ws[1]:
        txt = str(c.value).strip()
        if txt == "Context":
            c.value = "Term_Example"
        elif txt == "Note":
            c.value = "NoteImport"
    header = [str(c.value).strip() if c.value else "" for c in ws[1]]  # refresh

    # 3-d) map each Term_Example to its nearest Region/NoteImport/Usage note
    term_cols = [i for i, h in enumerate(header, 1) if h == "Term_Example"]
    left_region  = {}
    right_noteimp = {}
    right_usagenote = {}

    for tc in term_cols:
        # nearest Region to the left
        left_region[tc] = next((j for j in range(tc-1, 0, -1)
                               if header[j-1] == "Region"), None)
        # nearest NoteImport to the right
        right_noteimp[tc] = next((j for j in range(tc+1, len(header)+1)
                                 if header[j-1] == "NoteImport"), None)
        # nearest Usage note to the right
        right_usagenote[tc] = next((j for j in range(tc+1, len(header)+1)
                                   if header[j-1] == "Usage note"), None)

    # 3-e) row loop: replacements, highlighting, merging
    for row in ws.iter_rows(min_row=2, max_row=ws.max_row):
        # Mexico → Latin America
        for cell in row:
            if str(cell.value).strip() == "Mexico":
                cell.value = "Latin America"
                mex_total += 1

        # language highlighting
        for lang_col, usage_col in lang_to_usage.items():
            if usage_col and str(row[usage_col-1].value).strip().lower() == "notrecommended":
                tgt = row[lang_col-1]
                nf  = copy(tgt.font) if tgt.font else Font()
                nf.bold = False
                nf.color = RED_RGB
                tgt.font = nf
                hits_total += 1

        # merge into Term_Example
        for tc in term_cols:
            pieces = []
            lc = left_region[tc]
            rc1 = right_noteimp[tc]
            rc2 = right_usagenote[tc]

            for idx in (lc, rc1, rc2):
                if idx and row[idx-1].value not in (None, ""):
                    pieces.append(str(row[idx-1].value).strip())

            if pieces:
                existing = str(row[tc-1].value).strip() if row[tc-1].value else ""
                new_val  = ", ".join([p for p in [existing] + pieces if p])
                if new_val != existing:
                    row[tc-1].value = new_val
                    merged_rows += 1

print(f"✓ Highlighted {hits_total} cell(s)")
print(f"✓ Replaced {mex_total} 'Mexico' value(s)")
print(f"✓ Added Region/NoteImport/Usage note into Term_Example on {merged_rows} row(s)")

In [ ]:
# ── 4. Export one workbook per sheet (fixed) ───────────────────────
for ws in wb.worksheets:
    out = src.with_stem(f"{src.stem}-{ws.title}")
    new_wb = Workbook()
    new_ws = new_wb.active          # only one new workbook
    new_ws.title = ws.title

    # copy cell values plus red font
    for r in ws.iter_rows(values_only=False):
        for c in r:
            nc = new_ws.cell(row=c.row, column=c.column, value=c.value)
            if (c.font and c.font.color and c.font.color.type == "rgb"
                    and c.font.color.rgb[-6:].upper() == RED_RGB):
                nc.font = Font(color=RED_RGB)

    # copy simple column widths / row heights
    for col_letter, dim in ws.column_dimensions.items():
        new_ws.column_dimensions[col_letter].width = dim.width
    for idx, dim in ws.row_dimensions.items():
        new_ws.row_dimensions[idx].height = dim.height

    new_wb.save(out)
    print(f"✓ Saved → {out.name}")
    
print("All done!")